In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import math
import os
import time

2021-07-29 18:44:29.301748: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# get all train texts
train_texts = ""
files = os.listdir("../shahname/train")
for path in files:
	train_texts += open("../shahname/train/" + path, 'rb').read().decode(encoding='utf-8')

In [26]:
file_text = open("../allShahnameWords.txt", 'rb').read().decode(encoding = "utf-8")
vocab = sorted(set(file_text))
vocab.append(" ")

ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
vocab.append(" ")
vocab

In [26]:
# here we can get the chars of text like 'sdfsadf' and turn it into ids
# chars = tf.strings.unicode_split(train_texts, input_encoding='UTF-8')
# ids = ids_from_chars(chars)	

2021-07-28 05:04:47.670531: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 19409176 exceeds 10% of free system memory.
2021-07-28 05:04:48.068214: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 58227504 exceeds 10% of free system memory.
2021-07-28 05:04:48.370862: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 19409168 exceeds 10% of free system memory.


In [28]:
# turn ids into human readable text

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [29]:
all_ids = ids_from_chars(tf.strings.unicode_split(train_texts, 'UTF-8'))
all_ids

<tf.Tensor: shape=(2426146,), dtype=int64, numpy=array([24, 43, 36, ..., 22, 20,  1])>

In [30]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

س
ک
ن
د
ر
 
چ
و
 
ا


In [31]:
seq_length = 100
examples_per_epoch = len(train_texts)//(seq_length+1)

In [32]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  tmp = chars_from_ids(seq).numpy()
  print(''.join([str(s, encoding='UTF-8') for s in tmp]))

سکندر چو از کارش آگاه شد[UNK]که دارا به تخت افسر ماه شد
سپه برگرفت از عراق و براند[UNK]به رومی همی نام یزدان 


In [33]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode("utf-8"))

سکندر چو از کارش آگاه شد[UNK]که دارا به تخت افسر ماه شد
سپه برگرفت از عراق و براند[UNK]به رومی همی نام یزدان 
بخواند
سپه را میان و کرانه نبود[UNK]همان بخت دارا جوانه نبود
پذیره شدن را بیاراست شاه[UNK]بیاورد ز اصطخر چندا
ن سپاه
که گفتی ستاره نتابد همی[UNK]فلک راه رفتن نیابد همی
سپاه دو کشور کشیدند صف[UNK]همه نیزه و گرز و خنجر به
 کف
برآمد چنان از دو لشکر خروش[UNK]که چرخ فلک را بدرید گوش
چو دریا شد از خون گردان زمین[UNK]تن بی‌سران بد همه
 دشت کین
پدر را نبد بر پسر جای مهر[UNK]بریشان نبخشید گردان سپهر
سیم ره به دارا درآمد شکست[UNK]سکندر میان تاخت


In [34]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


dataset = sequences.map(split_input_target)

In [35]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy().decode("utf-8"))
    print("Target:", text_from_ids(target_example).numpy().decode("utf-8"))

Input : سکندر چو از کارش آگاه شد[UNK]که دارا به تخت افسر ماه شد
سپه برگرفت از عراق و براند[UNK]به رومی همی نام یزدان
Target: کندر چو از کارش آگاه شد[UNK]که دارا به تخت افسر ماه شد
سپه برگرفت از عراق و براند[UNK]به رومی همی نام یزدان 


In [36]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [37]:
# Model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.LSTM(rnn_units,
                                    #stateful=True 
                                   return_sequences=True,
                                   return_state=True
    )
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      h_state, c_state = self.gru.get_initial_state(x)
    else:
      h_state, c_state = states
    #x, states = self.gru(x, initial_state=states, training=training)
    x, h_state,c_state = self.gru(x, initial_state=[h_state,c_state], training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, h_state, c_state
    else:
      return x

In [39]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [40]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 48) # (batch_size, sequence_length, vocab_size)


In [41]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  12288     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_1 (Dense)              multiple                  49200     
Total params: 5,308,464
Trainable params: 5,308,464
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Befor training
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 48)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.8713398


In [43]:
tf.exp(mean_loss).numpy()

48.00666

In [44]:
model.compile(optimizer='adam', loss=loss)

## Training

In [ ]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

In [45]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # h_state, c_state = states
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, h_state, c_state = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, h_state, c_state

In [46]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode("utf-8"))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode("utf-8"))

Input:
 ه شهر اندرون سوفزای[UNK]بزرگان برفتند یک سر ز جای
پذیره شدن را بیاراست شاه[UNK]همی‌رفت با آنک بودش سپاه
بلاش

Next Char Predictions:
 غ(گعئ،[UNK]أاک«ح؟بٔ»(
طد(ئژ)ئق
عف[UNK]صژیو‌خشرمخمگاآرژخطسستئ،ٔجعقتشژل[UNK]شئ»»چات«ذوگ،چضم
شؤن‌بٔیء،ثرگل»صثل[UNK]ژپفح


In [47]:
# Save model
save_dir = "model-char-v3.h5"
# model.save_weights(save_dir)  # to store

# Load model
model.load_weights(save_dir)  # to load


In [48]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

## Generate fake texts

In [53]:
start = time.time()
next_char = tf.constant(['بگرییم چندی'])
states = None
zero_out = tf.keras.layers.Lambda(lambda x: tf.zeros_like(x))(next_char)
result = [next_char]

for n in range(200):
  next_char, h_state, c_state = one_step_model.generate_one_step(next_char, states=states)
  states = [h_state, c_state]
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


بگرییم چندی زبانم چو شیر
بهانه همی جست با اردوان ایمن از هرچه چیست
چنین گفت خسرو که شیرین بخورد
بدان گونه شادم که خود جفت اویین ز رنج
وگر گویی از رنج یابد رهایی گروه
برفتند هرکس که دیو آورید
بر او شد آنکس که دست 

________________________________________________________________________________

Run time: 0.9889492988586426
